<a href="https://colab.research.google.com/github/genkai10/Capstone_roughflow/blob/main/Accelerometer_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tsfresh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/95.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 9.1 MB/s eta 0:00:00


In [ ]:
!pip install boruta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 1.8 MB/s eta 0:00:00


In [ ]:
#import all libraries
import os
import pandas as pd
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute
import numpy as np
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import RandomForestClassifier
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Directory paths for fall and ADL sequences
fall_dir = '/content/drive/MyDrive/AccelerometerData/Fall'
adl_dir = '/content/drive/MyDrive/AccelerometerData/ADL'

# Function to load CSV files and return a list of dataframes
def load_csv_data(directory,classification):
    data = []
    for file_name in os.listdir(directory):
        if file_name.endswith('.csv'):
            file_path = os.path.join(directory, file_name)
            # Specify column names explicitly
            df = pd.read_csv(file_path, header=None, names=['time', 'SV_total', 'Ax', 'Ay', 'Az'])
            df['time'] = pd.to_datetime(df['time'], unit='ms')
            df['classification'] = classification
            data.append(df)
    return data

# Load CSV files for fall and ADL sequences
fall_data = load_csv_data(fall_dir,0)
adl_data = load_csv_data(adl_dir,1)

combined_data = fall_data + adl_data

In [ ]:
def add_sequence_id(df, sequence_id):
    df['sequence_id'] = sequence_id
    return df

time_series = [add_sequence_id(df, sequence_id=i) for i, df in enumerate(combined_data)]


In [ ]:
time_series

[                       time  SV_total        Ax        Ay        Az  \
 0   1969-12-31 23:59:59.998  1.024924  0.128718  0.995601  0.206588   
 1   1970-01-01 00:00:00.030  1.003218  0.166793  0.974967  0.167530   
 2   1970-01-01 00:00:00.061  1.022040  0.175391  0.991916  0.172934   
 3   1970-01-01 00:00:00.065  1.028459  0.152300  1.002479  0.171952   
 4   1970-01-01 00:00:00.094  1.006250  0.139035  0.979880  0.181778   
 ..                      ...       ...       ...       ...       ...   
 188 1970-01-01 00:00:05.272  1.094388  0.064113 -0.351519  1.034413   
 189 1970-01-01 00:00:05.275  1.075617  0.094328 -0.346851  1.013779   
 190 1970-01-01 00:00:05.306  1.059123  0.147387 -0.293546  1.006901   
 191 1970-01-01 00:00:05.310  1.042477  0.182515 -0.286668  0.985529   
 192 1970-01-01 00:00:05.341  1.086908  0.196762 -0.367731  1.003707   
 
      classification  sequence_id  
 0                 0            0  
 1                 0            0  
 2                 0      

In [ ]:
def extract_tsfresh_features(ts_df):
    print(ts_df.head())
    features = extract_features(ts_df, column_id="sequence_id", column_sort="time")
    return features

features = pd.concat([extract_tsfresh_features(df) for df in time_series])

                     time  SV_total        Ax        Ay        Az  \
0 1969-12-31 23:59:59.998  1.024924  0.128718  0.995601  0.206588   
1 1970-01-01 00:00:00.030  1.003218  0.166793  0.974967  0.167530   
2 1970-01-01 00:00:00.061  1.022040  0.175391  0.991916  0.172934   
3 1970-01-01 00:00:00.065  1.028459  0.152300  1.002479  0.171952   
4 1970-01-01 00:00:00.094  1.006250  0.139035  0.979880  0.181778   

   classification  sequence_id  
0               0            0  
1               0            0  
2               0            0  
3               0            0  
4               0            0  


Feature Extraction: 100%|██████████| 5/5 [00:00<00:00,  7.87it/s]


                     time  SV_total        Ax        Ay        Az  \
0 1969-12-31 23:59:59.994  1.014587  0.535507  0.830036  0.231644   
1 1970-01-01 00:00:00.041  1.007260  0.528875  0.826106  0.228942   
2 1970-01-01 00:00:00.050  1.013637  0.524944  0.835194  0.233118   
3 1970-01-01 00:00:00.082  1.037854  0.525927  0.862215  0.239013   
4 1970-01-01 00:00:00.113  1.028637  0.511434  0.859268  0.241224   

   classification  sequence_id  
0               0            1  
1               0            1  
2               0            1  
3               0            1  
4               0            1  


Feature Extraction: 100%|██████████| 5/5 [00:00<00:00, 11.53it/s]


                     time  SV_total        Ax        Ay        Az  \
0 1969-12-31 23:59:59.971  1.040439  0.578004  0.707459  0.497923   
1 1970-01-01 00:00:00.005  1.033213  0.581443  0.712372  0.471148   
2 1970-01-01 00:00:00.009  1.020717  0.636467  0.674788  0.425949   
3 1970-01-01 00:00:00.053  1.101317  0.673068  0.747008  0.449286   
4 1970-01-01 00:00:00.086  1.079454  0.658084  0.727602  0.450268   

   classification  sequence_id  
0               0            2  
1               0            2  
2               0            2  
3               0            2  
4               0            2  


...

Feature Extraction: 100%|██████████| 5/5 [00:00<00:00,  5.75it/s]


                     time  SV_total        Ax        Ay        Az  \
0 1969-12-31 23:59:59.910  1.101880 -0.073694  1.084033 -0.183251   
1 1969-12-31 23:59:59.972  1.215606 -0.017441  1.198258 -0.203886   
2 1969-12-31 23:59:59.977  1.137086 -0.048147  1.113511 -0.225257   
3 1970-01-01 00:00:00.007  1.091983 -0.055762  1.063153 -0.242943   
4 1970-01-01 00:00:00.039  1.135284 -0.060429  1.088701 -0.316146   

   classification  sequence_id  
0               1           67  
1               1           67  
2               1           67  
3               1           67  
4               1           67  


Feature Extraction: 100%|██████████| 5/5 [00:01<00:00,  4.24it/s]


                     time  SV_total        Ax        Ay        Az  \
0 1969-12-31 23:59:59.975  1.018869 -0.987793  0.219727  0.118652   
1 1969-12-31 23:59:59.983  0.991951 -0.960938  0.219727  0.110840   
2 1969-12-31 23:59:59.991  0.979602 -0.945312  0.235352  0.103027   
3 1969-12-31 23:59:59.999  0.962084 -0.933594  0.219727  0.075684   
4 1970-01-01 00:00:00.007  0.933191 -0.902832  0.223633  0.075684   

   classification  sequence_id  
0               1           68  
1               1           68  
2               1           68  
3               1           68  
4               1           68  


Feature Extraction: 100%|██████████| 5/5 [00:13<00:00,  2.60s/it]


In [ ]:
features

,SV_total__variance_larger_than_standard_deviation,SV_total__has_duplicate_max,SV_total__has_duplicate_min,SV_total__has_duplicate,SV_total__sum_values,SV_total__abs_energy,SV_total__mean_abs_change,SV_total__mean_change,SV_total__mean_second_derivative_central,SV_total__median,...,classification__fourier_entropy__bins_5,classification__fourier_entropy__bins_10,classification__fourier_entropy__bins_100,classification__permutation_entropy__dimension_3__tau_1,classification__permutation_entropy__dimension_4__tau_1,classification__permutation_entropy__dimension_5__tau_1,classification__permutation_entropy__dimension_6__tau_1,classification__permutation_entropy__dimension_7__tau_1,classification__query_similarity_count__query_None__threshold_0.0,classification__mean_n_absolute_max__number_of_maxima_7
0,0.0,0.0,0.0,0.0,215.144565,287.587787,0.157687,0.000323,0.000173,1.031047,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,0.0
1,0.0,0.0,0.0,0.0,145.361559,199.370196,0.179308,0.000720,-0.000029,1.028880,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,0.0
2,0.0,0.0,0.0,0.0,136.455766,213.994020,0.209271,0.000630,0.000047,1.079454,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,0.0
3,0.0,0.0,0.0,0.0,338.512824,385.147373,0.107130,0.000127,-0.000010,1.038740,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,0.0
4,0.0,0.0,0.0,0.0,222.334417,400.466888,0.274903,0.001022,-0.000379,1.043725,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0.0,0.0,0.0,1.0,1547.491610,1577.625731,0.023266,0.000095,0.000005,0.993912,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,1.0
65,0.0,0.0,0.0,1.0,1549.329089,1580.398254,0.022673,-0.000043,-0.000036,0.997738,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,1.0
66,0.0,0.0,0.0,0.0,351.820069,399.901768,0.069421,-0.000017,0.000158,1.116932,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,1.0
67,0.0,0.0,0.0,0.0,454.799159,523.435682,0.070389,-0.000037,-0.000152,1.094166,...,NaN,NaN,NaN,-0.0,-0.0,-0.0,-0.0,-0.0,NaN,1.0


In [ ]:
y = pd.Series([df['classification'][0] for df in time_series])

features.replace([np.inf, -np.inf], np.nan, inplace=True)

impute(features)
features_filtered = select_features(features, y)

/usr/local/lib/python3.10/dist-packages/tsfresh/utilities/dataframe_functions.py:198: RuntimeWarning: The columns ['SV_total__query_similarity_count__query_None__threshold_0.0'
 'Ax__query_similarity_count__query_None__threshold_0.0'
 'Ay__query_similarity_count__query_None__threshold_0.0'
 'Az__query_similarity_count__query_None__threshold_0.0'
 'classification__autocorrelation__lag_0'
 'classification__autocorrelation__lag_1'
 'classification__autocorrelation__lag_2'
 'classification__autocorrelation__lag_3'
 'classification__autocorrelation__lag_4'
 'classification__autocorrelation__lag_5'
 'classification__autocorrelation__lag_6'
 'classification__autocorrelation__lag_7'
 'classification__autocorrelation__lag_8'
 'classification__autocorrelation__lag_9'
 'classification__partial_autocorrelation__lag_1'
 'classification__partial_autocorrelation__lag_2'
 'classification__partial_autocorrelation__lag_3'
 'classification__partial_autocorrelation__lag_4'
 'classification__partial_autoco

In [ ]:
features_filtered

,classification__mean_n_absolute_max__number_of_maxima_7,"classification__cwt_coefficients__coeff_2__w_2__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_2__w_5__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_2__w_10__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_3__w_2__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_3__w_5__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_3__w_10__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_4__w_10__widths_(2, 5, 10, 20)",...,"Az__fft_aggregated__aggtype_""kurtosis""","Az__fft_coefficient__attr_""real""__coeff_95","SV_total__fft_coefficient__attr_""abs""__coeff_80","Ay__fft_coefficient__attr_""abs""__coeff_16","SV_total__change_quantiles__f_agg_""mean""__isabs_False__qh_0.4__ql_0.0","Ay__fft_coefficient__attr_""abs""__coeff_38","Ax__fft_coefficient__attr_""abs""__coeff_92",SV_total__count_below_mean,Az__median,Ay__quantile__q_0.9
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,8.269579,-1.447145,4.485553,7.395637,-0.013888,4.828358,2.102241,141.0,0.182515,1.123779
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,7.335085,-0.143353,1.396028,4.321444,-0.010073,1.223421,1.153246,104.0,0.269842,0.956986
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,6.641739,-0.143353,1.396028,4.875203,-0.022572,0.402328,1.153246,98.0,0.662506,0.819522
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,5.111684,-0.659080,5.577839,5.689109,-0.003950,1.743277,4.316382,225.0,0.165319,1.052738
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,13.039737,-9.194754,13.517261,5.433309,-0.023599,5.228768,8.096039,145.0,0.294775,1.065389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,1.0,0.759824,0.719559,0.538397,0.601882,0.976286,0.787617,0.328326,1.13112,1.013992,...,7.195812,-2.968675,14.176168,16.253792,0.000214,2.558777,12.554145,1047.0,0.025391,0.995117
65,1.0,0.759824,0.719559,0.538397,0.601882,0.976286,0.787617,0.328326,1.13112,1.013992,...,8.071858,0.643480,19.652374,18.434103,0.000068,5.490542,9.439369,951.0,0.044922,0.960449
66,1.0,0.759824,0.719559,0.538397,0.601882,0.976286,0.787617,0.328326,1.13112,1.013992,...,4.523999,-0.668786,1.333366,3.862977,-0.001630,1.814015,0.914139,157.0,-0.415878,1.085212
67,1.0,0.759824,0.719559,0.538397,0.601882,0.976286,0.787617,0.328326,1.13112,1.013992,...,4.645562,-1.317875,5.312049,9.300432,-0.004563,1.488959,2.835157,255.0,-0.358397,1.095775


In [ ]:
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Calculate mutual information scores
mi_scores = mutual_info_classif(features_filtered, y)

# Select top 500 features based on mutual information scores
k_best = SelectKBest(mutual_info_classif, k=500)
selected_features = k_best.fit_transform(features_filtered, y)

# Get the indices of the selected features
selected_indices = k_best.get_support(indices=True)

# Filter the original features dataframe to keep only the selected features
selected_features_df = features_filtered.iloc[:, selected_indices]


In [ ]:
selected_features_df

,classification__mean_n_absolute_max__number_of_maxima_7,"classification__cwt_coefficients__coeff_2__w_2__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_2__w_5__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_2__w_10__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_3__w_2__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_3__w_5__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_3__w_10__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_4__w_2__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_4__w_5__widths_(2, 5, 10, 20)","classification__cwt_coefficients__coeff_4__w_10__widths_(2, 5, 10, 20)",...,"SV_total__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""var""","SV_total__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""var""",Az__skewness,"Ax__agg_linear_trend__attr_""slope""__chunk_len_5__f_agg_""var""","Az__agg_linear_trend__attr_""intercept""__chunk_len_5__f_agg_""var""","SV_total__agg_linear_trend__attr_""slope""__chunk_len_10__f_agg_""max""","classification__fft_coefficient__attr_""angle""__coeff_70","classification__fft_coefficient__attr_""angle""__coeff_44",Az__time_reversal_asymmetry_statistic__lag_1,SV_total__lempel_ziv_complexity__bins_3
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.006715,0.018707,4.845216,0.005165,-0.003645,0.042582,0.000000,0.000000,0.116340,0.103627
1,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.013051,0.031584,5.575011,0.002687,0.043456,0.084055,0.000000,0.000000,0.374337,0.138462
2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,-0.008226,-0.015660,5.728065,-0.001124,0.428037,-0.012551,0.000000,0.000000,0.311670,0.153846
3,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000648,0.001496,1.087121,0.000706,-0.001582,0.004801,0.000000,0.000000,0.000850,0.104762
4,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.030532,0.065547,9.128598,0.027573,-0.052154,0.078907,0.000000,0.000000,-0.305197,0.100000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,1.0,0.759824,0.719559,0.538397,0.601882,0.976286,0.787617,0.328326,1.13112,1.013992,...,-0.000021,-0.000108,-1.169632,-0.000026,0.003244,-0.001154,122.875648,144.093264,-0.000030,0.071891
65,1.0,0.759824,0.719559,0.538397,0.601882,0.976286,0.787617,0.328326,1.13112,1.013992,...,-0.000013,-0.000067,-0.353551,-0.000016,0.002591,-0.000768,-179.883193,66.930565,-0.000037,0.079169
66,1.0,0.759824,0.719559,0.538397,0.601882,0.976286,0.787617,0.328326,1.13112,1.013992,...,0.000233,0.000380,-0.769503,0.000070,0.007837,-0.001031,0.000000,0.000000,-0.003934,0.107937
67,1.0,0.759824,0.719559,0.538397,0.601882,0.976286,0.787617,0.328326,1.13112,1.013992,...,0.000579,0.001207,-1.523931,0.000172,0.007530,0.003175,50.417690,165.405405,0.005727,0.073710


In [ ]:
# Step 1: Remove Highly Correlated Features Using Pearson Correlation Coefficient
def remove_highly_correlated_features(features_df, threshold=0.85):
    corr_matrix = features_df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    features_df_filtered = features_df.drop(columns=to_drop)
    return features_df_filtered

def apply_boruta(features_df, y):
    # Initialize Random Forest classifier
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    boruta_selector = BorutaPy(rf, n_estimators=100, verbose=2, random_state=42)
    boruta_selector.fit(features_df.values, y.values)
    selected_features = features_df.columns[boruta_selector.support_].tolist()
    return features_df[selected_features]


# Step 3: Selection of Top Features Using Feature Importance
def select_top_features(features_df, y, n_top_features=5):
    selector = SelectKBest(score_func=f_classif, k=n_top_features)
    selector.fit(features_df, y)
    selected_indices = selector.get_support(indices=True)
    selected_features = features_df.columns[selected_indices].tolist()
    return features_df[selected_features]

# Assuming features_df is your DataFrame containing all the features and y is your target variable

# Step 1: Remove Highly Correlated Features
features_df_filtered = remove_highly_correlated_features(selected_features_df)

# Step 2: Boruta Algorithm
#boruta_features = apply_boruta(features_df_filtered, y)

# Step 3: Selection of Top Features Using Feature Importance
top_features = select_top_features(features_df_filtered, y, n_top_features=10)


/usr/local/lib/python3.10/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: divide by zero encountered in divide
  f = msb / msw


In [ ]:
top_feature_headings = top_features.columns.tolist()
top_feature_headings

['classification__mean_n_absolute_max__number_of_maxima_7',
 'Ay__number_crossing_m__m_-1',
 'Ay__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',
 'SV_total__mean_n_absolute_max__number_of_maxima_7',
 'classification__ar_coefficient__coeff_1__k_10',
 'SV_total__ratio_beyond_r_sigma__r_0.5',
 'Az__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',
 'SV_total__energy_ratio_by_chunks__num_segments_10__segment_focus_8',
 'Ax__autocorrelation__lag_5',
 'Az__ratio_beyond_r_sigma__r_1']

In [ ]:
top_features

,classification__mean_n_absolute_max__number_of_maxima_7,Ay__number_crossing_m__m_-1,"Ay__change_quantiles__f_agg_""var""__isabs_False__qh_0.8__ql_0.0",SV_total__mean_n_absolute_max__number_of_maxima_7,classification__ar_coefficient__coeff_1__k_10,SV_total__ratio_beyond_r_sigma__r_0.5,"Az__change_quantiles__f_agg_""mean""__isabs_True__qh_1.0__ql_0.6",SV_total__energy_ratio_by_chunks__num_segments_10__segment_focus_8,Ax__autocorrelation__lag_5,Az__ratio_beyond_r_sigma__r_1
0,0.0,6.0,0.073931,2.727056,0.000000,0.207254,0.256759,0.085801,0.355334,0.108808
1,0.0,2.0,0.061786,2.697208,0.000000,0.146154,0.235528,0.076574,-0.104741,0.046154
2,0.0,6.0,0.153834,3.295551,0.000000,0.179487,0.170564,0.064470,0.405969,0.051282
3,0.0,4.0,0.065362,2.316426,0.000000,0.298413,0.118125,0.100028,0.174281,0.355556
4,0.0,12.0,0.175010,4.188468,0.000000,0.131579,0.197749,0.066194,0.225450,0.036842
...,...,...,...,...,...,...,...,...,...,...
64,1.0,0.0,0.001454,1.812557,0.090909,0.325777,0.026256,0.096677,0.971116,0.203368
65,1.0,0.0,0.001059,1.695596,-0.141174,0.419857,0.025348,0.097373,0.972091,0.255029
66,1.0,0.0,0.008906,1.755912,0.090909,0.368254,0.048751,0.099940,0.916321,0.438095
67,1.0,0.0,0.002814,2.070833,-0.025864,0.304668,0.038452,0.100428,0.879048,0.302211


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(top_features, y, test_size=0.2, random_state=42)

classifier = RandomForestClassifier(n_estimators=100, random_state=42)

classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
#print("Accuracy:", accuracy)

cv_scores = cross_val_score(classifier, top_features, y, cv=3)
average_accuracy = cv_scores.mean()
print("Average accuracy (k-fold cross-validation):", average_accuracy)


Average accuracy (k-fold cross-validation): 0.9710144927536232


In [ ]:
from joblib import dump
import os

# Define the path to save the model in Google Drive
drive_path = "/content/drive/MyDrive/"

# Save the classifier to Google Drive
dump(classifier, os.path.join(drive_path, "accelerometer_model.joblib"))
print("Model saved to Google Drive successfully.")

Model saved to Google Drive successfully.


In [ ]:
from joblib import load

model_path = "/content/drive/MyDrive/accelerometer_model.joblib"
classifier = load(model_path)

data_path = "/content/drive/MyDrive/synthetic_fall - Sheet1.csv"
test_df = pd.read_csv(data_path, header=None, names=['time', 'SV_total', 'Ax', 'Ay', 'Az'])
test_df['time'] = pd.to_datetime(test_df['time'], unit='ms')

test_df

,time,SV_total,Ax,Ay,Az
0,1969-12-31 23:59:59.999,1.019743,0.572470,0.749407,0.454094
1,1970-01-01 00:00:00.010,1.002518,0.623990,0.700537,0.436987
2,1970-01-01 00:00:00.058,1.085433,0.650644,0.768567,0.446828
3,1970-01-01 00:00:00.091,1.060454,0.647040,0.748101,0.447283
4,1970-01-01 00:00:00.125,1.020078,0.610928,0.731014,0.434456
...,...,...,...,...,...
111,1970-01-01 00:00:03.071,1.075801,0.361910,-0.565031,0.850242
112,1970-01-01 00:00:03.103,1.079797,0.342307,-0.585581,0.853891
113,1970-01-01 00:00:03.138,1.087549,0.350364,-0.565262,0.861942
114,1970-01-01 00:00:03.167,1.090011,0.394039,-0.551825,0.857476


In [ ]:
test_time_series = add_sequence_id(test_df, sequence_id=1)

In [ ]:
test_time_series

,time,SV_total,Ax,Ay,Az,sequence_id
0,1969-12-31 23:59:59.999,1.019743,0.572470,0.749407,0.454094,1
1,1970-01-01 00:00:00.010,1.002518,0.623990,0.700537,0.436987,1
2,1970-01-01 00:00:00.058,1.085433,0.650644,0.768567,0.446828,1
3,1970-01-01 00:00:00.091,1.060454,0.647040,0.748101,0.447283,1
4,1970-01-01 00:00:00.125,1.020078,0.610928,0.731014,0.434456,1
...,...,...,...,...,...,...
111,1970-01-01 00:00:03.071,1.075801,0.361910,-0.565031,0.850242,1
112,1970-01-01 00:00:03.103,1.079797,0.342307,-0.585581,0.853891,1
113,1970-01-01 00:00:03.138,1.087549,0.350364,-0.565262,0.861942,1
114,1970-01-01 00:00:03.167,1.090011,0.394039,-0.551825,0.857476,1


In [ ]:
test_features = extract_tsfresh_features(test_time_series)
test_features

                     time  SV_total        Ax        Ay        Az  sequence_id
0 1969-12-31 23:59:59.999  1.019743  0.572470  0.749407  0.454094            1
1 1970-01-01 00:00:00.010  1.002518  0.623990  0.700537  0.436987            1
2 1970-01-01 00:00:00.058  1.085433  0.650644  0.768567  0.446828            1
3 1970-01-01 00:00:00.091  1.060454  0.647040  0.748101  0.447283            1
4 1970-01-01 00:00:00.125  1.020078  0.610928  0.731014  0.434456            1


Feature Extraction: 100%|██████████| 4/4 [00:00<00:00, 10.09it/s]


,SV_total__variance_larger_than_standard_deviation,SV_total__has_duplicate_max,SV_total__has_duplicate_min,SV_total__has_duplicate,SV_total__sum_values,SV_total__abs_energy,SV_total__mean_abs_change,SV_total__mean_change,SV_total__mean_second_derivative_central,SV_total__median,...,Az__fourier_entropy__bins_5,Az__fourier_entropy__bins_10,Az__fourier_entropy__bins_100,Az__permutation_entropy__dimension_3__tau_1,Az__permutation_entropy__dimension_4__tau_1,Az__permutation_entropy__dimension_5__tau_1,Az__permutation_entropy__dimension_6__tau_1,Az__permutation_entropy__dimension_7__tau_1,Az__query_similarity_count__query_None__threshold_0.0,Az__mean_n_absolute_max__number_of_maxima_7
1,0.0,0.0,0.0,0.0,133.251135,206.326448,0.207743,0.000639,0.00009,1.061452,...,1.089718,1.646216,3.222606,1.718351,2.91961,3.993427,4.512765,4.675275,NaN,2.687978


In [ ]:
# selected_test_features = pd.DataFrame()
# selected_test_features['classification__mean_n_absolute_max__number_of_maxima_7'] = 0
# selected_test_features = test_features[["Ay__number_crossing_m__m_-1",'Ay__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0','SV_total__mean_n_absolute_max__number_of_maxima_7']]
# selected_test_features['classification__ar_coefficient__coeff_1__k_10'] = 0
# selected_test_features = test_features[['SV_total__ratio_beyond_r_sigma__r_0.5',
#  'Az__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',
#  'SV_total__energy_ratio_by_chunks__num_segments_10__segment_focus_8',
#  'Ax__autocorrelation__lag_5',
#  'Az__ratio_beyond_r_sigma__r_1']]
# Create a new DataFrame with specific column order
# Create a list of columns to add
columns_to_add = ['classification__mean_n_absolute_max__number_of_maxima_7',
                  'Ay__number_crossing_m__m_-1',
                  'Ay__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',
                  'SV_total__mean_n_absolute_max__number_of_maxima_7',
                  'classification__ar_coefficient__coeff_1__k_10',
                  'SV_total__ratio_beyond_r_sigma__r_0.5',
                  'Az__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',
                  'SV_total__energy_ratio_by_chunks__num_segments_10__segment_focus_8',
                  'Ax__autocorrelation__lag_5',
                  'Az__ratio_beyond_r_sigma__r_1']

# Check if any columns are missing in the original DataFrame
missing_columns = [col for col in columns_to_add if col not in test_features.columns]

# Add missing columns to the original DataFrame
for col in missing_columns:
    test_features[col] = 0

# Create a new DataFrame with specific column order
selected_test_features = test_features[columns_to_add]



In [ ]:
selected_test_features.columns

Index(['classification__mean_n_absolute_max__number_of_maxima_7',
       'Ay__number_crossing_m__m_-1',
       'Ay__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',
       'SV_total__mean_n_absolute_max__number_of_maxima_7',
       'classification__ar_coefficient__coeff_1__k_10',
       'SV_total__ratio_beyond_r_sigma__r_0.5',
       'Az__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',
       'SV_total__energy_ratio_by_chunks__num_segments_10__segment_focus_8',
       'Ax__autocorrelation__lag_5', 'Az__ratio_beyond_r_sigma__r_1'],
      dtype='object')

In [ ]:
top_feature_headings

['classification__mean_n_absolute_max__number_of_maxima_7',
 'Ay__number_crossing_m__m_-1',
 'Ay__change_quantiles__f_agg_"var"__isabs_False__qh_0.8__ql_0.0',
 'SV_total__mean_n_absolute_max__number_of_maxima_7',
 'classification__ar_coefficient__coeff_1__k_10',
 'SV_total__ratio_beyond_r_sigma__r_0.5',
 'Az__change_quantiles__f_agg_"mean"__isabs_True__qh_1.0__ql_0.6',
 'SV_total__energy_ratio_by_chunks__num_segments_10__segment_focus_8',
 'Ax__autocorrelation__lag_5',
 'Az__ratio_beyond_r_sigma__r_1']

In [ ]:
predictions = classifier.predict(selected_test_features)
confidences = classifier.predict_proba(selected_test_features)[:, 0]
print (predictions)
print (confidences)
# Convert predictions to human-readable labels if necessary
predicted_labels = ["Fall" if pred == 0 else "Not Fall" for pred in predictions]

# Print the predictions
print("Predictions for the new test data:")
for label in predicted_labels:
    print(label)


[0]
[1.]
Predictions for the new test data:
Fall


In [ ]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
def generate_labels_dict(video_dir, label_mapping):
    labels = {}
    for label_name, label_index in label_mapping.items():
        label_path = os.path.join(video_dir, label_name)
        if os.path.exists(label_path):
            for video_file in os.listdir(label_path):
                if video_file.endswith('.mp4'):
                    labels[os.path.join(label_name, video_file)] = label_index
    return labels

label_mapping = {
    'sitting': 0,
    'eating': 1,
    'fall': 2,
    'laydown': 3,
    'walking': 4,
}

# Define the path to your dataset directory
video_dir = '/content/drive/MyDrive/smarthome'
labels = generate_labels_dict(video_dir, label_mapping)

class VideoDataset(Dataset):
    def __init__(self, video_dir, labels, transform=None, sequence_length=16):
        self.video_dir = video_dir
        self.labels = labels
        self.transform = transform
        self.sequence_length = sequence_length
        self.video_files = list(labels.keys())

    def __len__(self):
        return len(self.video_files)

    def __getitem__(self, idx):
        video_file = self.video_files[idx]
        video_path = os.path.join(self.video_dir, video_file)
        label = self.labels[video_file]

        cap = cv2.VideoCapture(video_path)
        frames = []
        while len(frames) < self.sequence_length:
            ret, frame = cap.read()
            if not ret:
                break
            if self.transform:
                frame = self.transform(frame)
            frames.append(frame)
        cap.release()

        while len(frames) < self.sequence_length:
            frames.append(frames[-1])

        video_tensor = torch.stack(frames)
        return video_tensor, label

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

dataset = VideoDataset(video_dir, labels, transform=transform, sequence_length=16)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:
import torch
class TransformerModel(torch.nn.Module):
    # Model definition (from previous parts)
    def __init__(self, feature_dim, num_classes, num_layers, num_heads, dropout):
        super(TransformerModel, self).__init__()
        self.feature_extractor = torchvision.models.resnet18(pretrained=True)
        self.feature_extractor.fc = torch.nn.Identity()  # Remove the final fully connected layer
        self.transformer = torch.nn.Transformer(
            d_model=feature_dim,
            nhead=num_heads,
            num_encoder_layers=num_layers,
            num_decoder_layers=num_layers,
            dropout=dropout
        )
        self.fc = torch.nn.Linear(feature_dim, num_classes)

    def forward(self, x):
        batch_size, seq_len, c, h, w = x.size()
        x = x.view(batch_size * seq_len, c, h, w)
        features = self.feature_extractor(x)
        features = features.view(batch_size, seq_len, -1).permute(1, 0, 2)
        transformer_output = self.transformer(features, features)
        output = self.fc(transformer_output.mean(dim=0))
        return output
trans_model = torch.load('/content/drive/MyDrive/smarthome/transformer_model_complete.pth')
trans_model.eval()
def evaluate(model, dataloader):
    video_predictions = []
    video_confidences = []
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for videos, labels in dataloader:
            videos = videos.to(device)
            labels = labels.to(device)
            outputs = model(videos)
            confidence, predicted = torch.max(outputs, 1)
            video_predictions.extend(predicted.cpu().numpy())
            video_confidences.extend(confidence.cpu().numpy())
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = correct / total
    print(f'Accuracy: {accuracy:.4f}')
    return video_predictions, video_confidences
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
test_video_dir= '/content/drive/MyDrive/smarthome/test'
test_labels = generate_labels_dict(test_video_dir, label_mapping)
print(test_labels)
# Assuming you have a test dataloader
test_dataset = VideoDataset(test_video_dir, test_labels, transform=transform, sequence_length=16)
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)
video_predictions, video_confidences = evaluate(trans_model, test_dataloader)

{'sitting/Sitdown_p06_r02_v14_c01.mp4': 0, 'sitting/Sitdown_p06_r02_v10_c01.mp4': 0, 'sitting/Sitdown_p06_r02_v11_c02.mp4': 0, 'sitting/Sitdown_p07_r01_v07_c04.mp4': 0, 'sitting/Sitdown_p06_r03_v05_c04.mp4': 0, 'eating/Eat.Snack_p04_r00_v16_c06.mp4': 1, 'eating/Eat.Snack_p06_r00_v06_c04.mp4': 1, 'eating/Eat.Attable_p18_r17_v09_c02.mp4': 1, 'eating/Eat.Attable_p18_r14_v09_c02.mp4': 1, 'eating/Eat.Attable_p18_r16_v09_c01.mp4': 1, 'fall/Copy of fall-07-cam0.mp4': 2, 'fall/Copy of fall-11-cam0.mp4': 2, 'fall/Copy of fall-29-cam0.mp4': 2, 'fall/Copy of fall-28-cam0.mp4': 2, 'fall/Copy of fall-23-cam0.mp4': 2, 'laydown/Laydown_p17_r01_v05_c05.mp4': 3, 'laydown/Laydown_p17_r02_v05_c04.mp4': 3, 'laydown/Laydown_p17_r04_v05_c04.mp4': 3, 'laydown/Laydown_p14_r05_v02_c05.mp4': 3, 'laydown/Laydown_p16_r01_v05_c04.mp4': 3, 'walking/Walk_p03_r17_v04_c04.mp4': 4, 'walking/Walk_p03_r39_v04_c05.mp4': 4, 'walking/Walk_p03_r40_v02_c04.mp4': 4, 'walking/Walk_p03_r18_v02_c05.mp4': 4, 'walking/Walk_p03_r18_

In [ ]:
accel_confidences = confidences
accel_predictions = predictions
final_predictions = []
for video_pred, video_conf, accel_pred, accel_conf in zip(video_predictions, video_confidences, accel_predictions, accel_confidences):
    if accel_pred == 0:  # Fall detected by accelerometer
        if accel_conf > 0.5:  # High confidence in fall detection
            final_predictions.append(2)  # 2 corresponds to 'fall' in the label mapping
        else:
            final_predictions.append(video_pred)  # Use video prediction if confidence is low
    else:
        if video_pred == 2 and video_conf > 0.5:  # Fall predicted by video model
            final_predictions.append(2)  # Confirm fall by video model
        else:
            final_predictions.append(video_pred)
print(final_predictions)

[2]
